In [ ]:
import re
import logging
from datetime import datetime, timedelta
import pendulum

In [ ]:
local_timezone_string = "US/Central"
current_timezone = pendulum.timezone(local_timezone_string)

In [ ]:
def get_execution_date_weekend(execution_date, schedule, special_check):  
    print("Execution Date - ",execution_date)  
    exec_day = int(execution_date.strftime('%w'))
    print("Execution Day - ",exec_day)
    schedule_day = schedule.split()[4]
    if special_check == "noweekend" and exec_day in [0, 5, 6]:
            schedule_day = 4
    else:
        if re.search(",",schedule_day):
            schedule_day=schedule_day.split(",")
            if str(exec_day) in schedule_day:
                schedule_day=exec_day
            else:
                logging.info("Dependant DAG is not scheduled to run today")
                print("Error")
                return
    if schedule_day == "*":
        poke_date = execution_date
    elif exec_day < int(schedule_day):
        date_diff = 7-int(schedule_day)+exec_day
        poke_date = execution_date - timedelta(days=date_diff)        
    elif exec_day >= int(schedule_day):
        date_diff = exec_day-int(schedule_day)
        poke_date = execution_date - timedelta(days=date_diff)
    year = int(poke_date.year)
    month = int(poke_date.month)
    date = int(poke_date.day)
    min = int(schedule.split()[0])
    hour = int(schedule.split()[1])
    poke_date = pendulum.datetime(year,month,date,hour,min,0,tz=current_timezone)
    print(poke_date)
    return poke_date

In [ ]:
execution_date = datetime(year=2024, month=3, day=11, hour=13, minute=0, second=0)
special_check = "noweekend"
get_execution_date_weekend(execution_date, "0 8 * * 1,2,3,4,5", special_check)

In [ ]:
def get_time_diff(starting_hour, starting_min, ending_hour, ending_minute):

    if starting_hour == ending_hour:
        hour_diff = 0
    elif starting_hour < ending_hour:
        hour_diff = ending_hour - starting_hour
    else:
        hour_diff = 24 - starting_hour + ending_hour

    if starting_min == ending_minute:
        min_diff = 0
    elif starting_min < ending_minute:
        min_diff = ending_minute - starting_min
    else:
        min_diff = 60 - starting_min + ending_minute
        hour_diff = hour_diff - 1

    return hour_diff, min_diff

In [ ]:
def get_local_datetime(year,month,date,hour,min):
    utc_date = pendulum.datetime(year,month,date,hour,min,0)
    local_date = current_timezone.convert(utc_date)
    print('Local Date : ', local_date)
    year = int(local_date.year)
    month = int(local_date.month)
    date = int(local_date.day)
    min = int(local_date.minute)
    hour = int(local_date.hour)
    return pendulum.datetime(year,month,date,hour,min,0,tz=current_timezone)

In [ ]:
def get_date_elements(date_time_value):
    year = int(date_time_value.year)
    month = int(date_time_value.month)
    date = int(date_time_value.day)
    return year, month, date

In [ ]:
def get_time_elements(date_time_value):
    hour = int(date_time_value.hour)
    min = int(date_time_value.minute)
    return hour, min

In [ ]:
def get_date_time_elements(date_time_value):
    year, month, date = get_date_elements(date_time_value)
    hour, min = get_time_elements(date_time_value)
    return year, month, date, hour, min

In [ ]:
def get_execution_date(execution_date, **kwargs):  
    print("Execution Date - ",execution_date)  
    exec_day = int(execution_date.strftime('%w'))
    schedule = kwargs["params"]["schedule"]
    schedule_day = schedule.split()[4]
    if re.search(",",schedule_day):
        schedule_day=schedule_day.split(",")
        if str(exec_day) in schedule_day:
            schedule_day=exec_day
        else:
            logging.info("Dependant DAG is not scheduled to run today")
            raise SystemExit()
    
    if schedule_day == "*":
        poke_date = execution_date
    elif exec_day < int(schedule_day):
        date_diff = 7-int(schedule_day)+exec_day
        poke_date = execution_date - timedelta(days=date_diff)        
    elif exec_day >= int(schedule_day):
        date_diff = exec_day-int(schedule_day)
        poke_date = execution_date - timedelta(days=date_diff)

    if "frequency" in kwargs["params"]:
        frequency = kwargs["params"]["frequency"]
    else:
        frequency = None

    if frequency is None or "hourly" not in frequency:
            scheduled_hour = int(schedule.split()[1])
            scheduled_min = int(schedule.split()[0])
            if "sensor_dag_frequency" in kwargs["params"]:
                sensor_dag_frequency = kwargs["params"]["sensor_dag_frequency"]
                if sensor_dag_frequency == "hourly":
                    if frequency is not None and frequency == "daily":
                        poke_date = poke_date.add(days=1)
                    year, month, date, hour, min = get_date_time_elements(poke_date)
                    poke_date_local = get_local_datetime(year,month,date,hour,min)
                    year, month, date, hour, min = get_date_time_elements(poke_date_local)
                    hour_diff, min_diff = get_time_diff(scheduled_hour, scheduled_min, hour, min)
                    if "hour_interval" in kwargs["params"]:
                        hour_interval = kwargs["params"]["hour_interval"]
                    else:
                        hour_interval = 0
                    prev_poke_date_local = poke_date_local.subtract(hours=hour_interval + hour_diff, minutes=min_diff)
                    year, month, date, hour, min = get_date_time_elements(prev_poke_date_local)
                elif sensor_dag_frequency == "daily":
                    if frequency is not None and frequency == "monthly":
                        poke_date = poke_date.add(months=1)
                    year, month, date, hour, min = get_date_time_elements(poke_date)
                    poke_date_local = get_local_datetime(year, month, date, hour, min)
                    year, month, date, hour, min = get_date_time_elements(poke_date_local)
                    hour_diff, min_diff = get_time_diff(scheduled_hour, scheduled_min, hour, min)
                    if frequency is None or frequency == "daily":
                        day_interval = 0
                    else:
                        if "day_interval" in kwargs["params"]:
                            day_interval = kwargs["params"]["day_interval"]
                        else:
                            day_interval = 0
                    prev_poke_date_local = poke_date_local.subtract(days=day_interval, hours=hour_diff, minutes=min_diff)
                    year, month, date, hour, min = get_date_time_elements(prev_poke_date_local)
            else:
                year, month, date = get_date_elements(poke_date)
                hour = scheduled_hour
                min = scheduled_min
    else:
        poke_date_local = current_timezone.convert(poke_date)
        print('Local Date : ', poke_date_local)
        if "cycle_age" in kwargs["params"]:
            cycle_age = kwargs["params"]["cycle_age"]
            if cycle_age == "previous":
                schedule_hour = schedule.split()[1]
                hourset_regex = re.compile("[\,]+", re.IGNORECASE)
                result = hourset_regex.search(schedule_hour)
                if result is None:
                    hourrange_regex = re.compile("([0-9]+)-([0-9]+)", re.IGNORECASE)
                    result = hourrange_regex.match(schedule_hour)
                    if result is None:
                        hourrange_regex = re.compile("([0-9]+)", re.IGNORECASE)
                        result = hourrange_regex.match(schedule_hour)
                        if result is None:
                            start_hour = 0
                        else:
                            start_hour = int(result.groups()[0])
                        end_hour = 24
                    else:
                        start_hour = int(result.groups()[0])
                        end_hour = int(result.groups()[1]) + 1
                    
                    hourstep_regex = re.compile("[0-9\-\*]+/([0-9]+)", re.IGNORECASE)
                    result = hourstep_regex.match(schedule_hour)
                    if result is None:
                        step_hour = 1
                    else:
                        step_hour = int(result.groups()[0])
                    scheduled_hour_list = []
                    for hour in range(start_hour, end_hour, step_hour):
                        scheduled_hour_list.append(hour)
                else:
                    scheduled_hour_list = list(map(int,schedule_hour.split(',')))
                scheduled_hour_list.insert(0, scheduled_hour_list.pop())

                poke_date_local_hour = int(poke_date_local.hour)
                poke_date_utc_offset = poke_date_local.utcoffset()
                poke_index = scheduled_hour_list.index(poke_date_local_hour)
                poke_date_local_hour_prev = scheduled_hour_list[poke_index-1]

                if poke_date_local_hour_prev > poke_date_local_hour:
                    prev_poke_hour_diff = 24 - poke_date_local_hour_prev + poke_date_local_hour
                else:
                    prev_poke_hour_diff = poke_date_local_hour - poke_date_local_hour_prev

                prev_date_date_local = poke_date_local.subtract(hours=prev_poke_hour_diff)

                prev_poke_date_utc_offset = prev_date_date_local.utcoffset()

                utc_offset_diff = prev_poke_date_utc_offset - poke_date_utc_offset

                prev_date_date_local = prev_date_date_local - utc_offset_diff

                year, month, date, hour, min = get_date_time_elements(prev_date_date_local)

            else:
                year, month, date, hour, min = get_date_time_elements(poke_date_local)
        else:
            year, month, date, hour, min = get_date_time_elements(poke_date_local)

    poke_date = pendulum.datetime(year,month,date,hour,min,0,tz=current_timezone)
    print(poke_date)
    return poke_date

In [ ]:
# Get Scheduled Interval Start Time for previous hourly cycle
execution_date_cst = pendulum.datetime(year=2024, month=3, day=10, hour=11, minute=0, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 4-18/2 * * *", "frequency" : "hourly", "cycle_age" : "previous"})
# Execution Date CST - 2024-03-09T10:00:00+00:00 - Poke Date Local - 2024-03-09T04:00:00-06:00 - Poke Date Local Prev - 2024-03-08T18:00:00-06:00
# Execution Date CST - 2024-03-09T22:00:00+00:00 - Poke Date Local - 2024-03-09T16:00:00-06:00 - Poke Date Local Prev - 2024-03-09T14:00:00-06:00
# Execution Date CST - 2024-03-10T00:00:00+00:00 - Poke Date Local - 2024-03-09T18:00:00-06:00 - Poke Date Local Prev - 2024-03-09T16:00:00-06:00
# Execution Date CST - 2024-03-10T09:00:00+00:00 - Poke Date Local - 2024-03-10T04:00:00-05:00 - Poke Date Local Prev - 2024-03-09T18:00:00-06:00
# Execution Date CST - 2024-03-10T11:00:00+00:00 - Poke Date Local - 2024-03-10T06:00:00-05:00 - Poke Date Local Prev - 2024-03-10T04:00:00-05:00

In [ ]:
# Get Scheduled Interval Start Time for prior hourly job from a daily job execution
execution_date_cst = pendulum.datetime(year=2024, month=3, day=10, hour=23, minute=0, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 18 * * *", "frequency" : "daily", "sensor_dag_frequency" : "hourly", 
"hour_interval" : 2})
# Execution Date CST - 2024-03-09T00:00:00+00:00 - Poke Date Local - 2024-03-09T16:00:00-06:00
# Execution Date CST - 2024-03-10T00:00:00+00:00 - Poke Date Local - 2024-03-10T16:00:00-05:00
# Execution Date CST - 2024-03-10T23:00:00+00:00 - Poke Date Local - 2024-03-11T16:00:00-05:00

In [ ]:
# Get Scheduled Interval Start Time for prior daily job from a monthly job execution
execution_date_cst = pendulum.datetime(year=2024, month=3, day=1, hour=6, minute=5, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 18 * * *", "frequency" : "monthly", "sensor_dag_frequency" : "daily", 
"day_interval" : 1})
# Execution Date UTC - 2024-02-01T06:05:00+00:00 - Poke Date CST - 2024-02-28T18:00:00-06:00
# Execution Date UTC - 2024-03-01T06:05:00+00:00 - Poke Date CST - 2024-03-30T18:00:00-05:00
# Execution Date UTC - 2024-04-01T05:05:00+00:00 - Poke Date CST - 2024-04-29T18:00:00-05:00

In [ ]:
# Get Scheduled Interval Start Time for daily cycle
execution_date_cst = pendulum.datetime(year=2024, month=3, day=10, hour=9, minute=0, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 4 * * *", "frequency" : "daily"})
# Execution Date UTC - 2024-03-09T10:00:00+00:00 - Poke Date CST - 2024-03-09T04:00:00-06:00
# Execution Date UTC - 2024-03-10T09:00:00+00:00 - Poke Date CST - 2024-03-10T04:00:00-05:00

In [ ]:
# Get Scheduled Interval Start Time for current hourly cycle
execution_date_cst = pendulum.datetime(year=2024, month=3, day=10, hour=9, minute=0, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 4-18/2 * * *", "frequency" : "hourly"})
# Execution Date UTC - 2024-03-09T22:00:00+00:00 - Poke Date CST - 2024-03-09T16:00:00-06:00
# Execution Date UTC - 2024-03-10T00:00:00+00:00 - Poke Date CST - 2024-03-09T18:00:00-06:00
# Execution Date UTC - 2024-03-10T09:00:00+00:00 - Poke Date CST - 2024-03-10T04:00:00-05:00

In [ ]:
# Get Scheduled Interval Start Time for prior hourly job from a daily job execution
execution_date_cst = pendulum.datetime(year=2025, month=7, day=6, hour=11, minute=45, second=0)
get_execution_date(execution_date_cst, params = {"schedule" : "0 8 6 * *", "frequency" : "daily", "sensor_dag_frequency" : "monthly"})
# Execution Date CST - 2024-03-09T00:00:00+00:00 - Poke Date Local - 2024-03-09T16:00:00-06:00
# Execution Date CST - 2024-03-10T00:00:00+00:00 - Poke Date Local - 2024-03-10T16:00:00-05:00
# Execution Date CST - 2024-03-10T23:00:00+00:00 - Poke Date Local - 2024-03-11T16:00:00-05:00